In [ ]:
# Dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# File paths and Tensorflow
import os
import tensorflow as tf

# Sklearn scaling
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

# Random forest model imports
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import seaborn as sns

# Keras
from tensorflow import keras
# from tensorflow.keras

In [ ]:
# Read in the liver_clean.csv

url = "https://raw.githubusercontent.com/amydohlin/project-4-group-2/main/cleaned_csvs/liver_clean.csv"

liver_clean_df = pd.read_csv(url)
liver_clean_df

,Unnamed: 0,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,0,2221,C,Placebo,18499,F,N,Y,N,N,0.5,149.000000,4.04,227.000000,598.000000,52.700000,57.000000,256.0,9.9,1
1,1,1230,C,Placebo,19724,M,Y,N,Y,N,0.5,219.000000,3.93,22.000000,663.000000,45.000000,75.000000,220.0,10.8,2
2,2,4184,C,Placebo,11839,F,N,N,N,N,0.5,320.000000,3.54,51.000000,1243.000000,122.450000,80.000000,225.0,10.0,2
3,3,2090,D,Placebo,16467,F,N,N,N,N,0.7,255.000000,3.74,23.000000,1024.000000,77.500000,58.000000,151.0,10.2,2
4,4,2105,D,Placebo,21699,F,N,Y,N,N,1.9,486.000000,3.54,74.000000,1052.000000,108.500000,109.000000,151.0,11.5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9634,24963,3577,C,Placebo,17897,F,Y,N,Y,N,0.7,369.510563,3.49,97.648387,1982.655769,122.556346,124.702128,243.0,9.7,1
9635,24971,4795,C,Placebo,23376,F,Y,N,Y,N,1.8,369.510563,3.24,97.648387,1982.655769,122.556346,124.702128,139.0,10.5,1
9636,24972,3358,D,D-penicillamine,24585,F,N,Y,N,N,2.1,262.000000,3.48,58.000000,2045.000000,89.900000,84.000000,412.0,11.8,3
9637,24991,4365,C,D-penicillamine,21324,F,N,N,N,N,0.9,346.000000,3.40,81.000000,1098.000000,122.450000,90.000000,228.0,10.3,2


In [ ]:
#drop the "unnamed: 0" uneccesary index column
liver_clean_df=liver_clean_df.drop(["Unnamed: 0"],axis=1)

liver_clean_df.head()

,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,2221,C,Placebo,18499,F,N,Y,N,N,0.5,149.0,4.04,227.0,598.0,52.70,57.0,256.0,9.9,1
1,1230,C,Placebo,19724,M,Y,N,Y,N,0.5,219.0,3.93,22.0,663.0,45.00,75.0,220.0,10.8,2
2,4184,C,Placebo,11839,F,N,N,N,N,0.5,320.0,3.54,51.0,1243.0,122.45,80.0,225.0,10.0,2
3,2090,D,Placebo,16467,F,N,N,N,N,0.7,255.0,3.74,23.0,1024.0,77.50,58.0,151.0,10.2,2
4,2105,D,Placebo,21699,F,N,Y,N,N,1.9,486.0,3.54,74.0,1052.0,108.50,109.0,151.0,11.5,1


In [ ]:
# Define list of columns of interest
desired_columns = ['Stage', 'Status', 'Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']

# Extract these columns
columns_of_interest = [col for col in liver_clean_df.columns if col in desired_columns]

#Print list
print(columns_of_interest)

['Status', 'Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Stage']


In [ ]:
# Check number of unique values in each column
liver_clean_df[columns_of_interest].nunique()

Status          3
Drug            2
Sex             2
Ascites         2
Hepatomegaly    2
Spiders         2
Edema           3
Stage           3
dtype: int64

In [ ]:
# Create OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform OneHotEncoder using categorical variable list
# Result is stored in new DataFrame called encode_df
encode_df = pd.DataFrame(enc.fit_transform(liver_clean_df[columns_of_interest]))

# Add encoded variable names to encode_df dataframe
# Assign encoded variable names to columns of encode_df DataFrame
# get_feature_names_out method retrieves feature names for encoded categorical variables
encode_df.columns = enc.get_feature_names_out(columns_of_interest)

#print
encode_df.head()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Status_C,Status_CL,Status_D,Drug_D-penicillamine,Drug_Placebo,Sex_F,Sex_M,Ascites_N,Ascites_Y,Hepatomegaly_N,Hepatomegaly_Y,Spiders_N,Spiders_Y,Edema_N,Edema_S,Edema_Y,Stage_1,Stage_2,Stage_3
0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
#Merge one-hot encoded features and drop original features

#Merge encoded features, encode_df, with original dataframe, liver_clean_df, into liver_clean_df
#left_index=True and right_index=True parameters indicate that merge is based on index of DataFrames
liver_clean_df = liver_clean_df.merge(encode_df, left_index=True, right_index=True)

#Drop original categorical columns, collected in liver_lean_cat, from liver_clean_df
#axis=1 parameter specifies columns (not rows) should be dropped
liver_clean_df = liver_clean_df.drop(columns_of_interest, axis=1)

liver_clean_df.head()

,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,...,Hepatomegaly_N,Hepatomegaly_Y,Spiders_N,Spiders_Y,Edema_N,Edema_S,Edema_Y,Stage_1,Stage_2,Stage_3
0,2221,18499,0.5,149.0,4.04,227.0,598.0,52.70,57.0,256.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,1230,19724,0.5,219.0,3.93,22.0,663.0,45.00,75.0,220.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
2,4184,11839,0.5,320.0,3.54,51.0,1243.0,122.45,80.0,225.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,2090,16467,0.7,255.0,3.74,23.0,1024.0,77.50,58.0,151.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,2105,21699,1.9,486.0,3.54,74.0,1052.0,108.50,109.0,151.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
#Check columns
print("Columns in DataFrame:")
print(liver_clean_df.columns.tolist())

Columns in DataFrame:
['N_Days', 'Age', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin', 'Status_C', 'Status_CL', 'Status_D', 'Drug_D-penicillamine', 'Drug_Placebo', 'Sex_F', 'Sex_M', 'Ascites_N', 'Ascites_Y', 'Hepatomegaly_N', 'Hepatomegaly_Y', 'Spiders_N', 'Spiders_Y', 'Edema_N', 'Edema_S', 'Edema_Y', 'Stage_1', 'Stage_2', 'Stage_3']


In [ ]:
# Get number of columns (features)
num_features = liver_clean_df.shape[1]
print("Number of features in the DataFrame:", num_features)

Number of features in the DataFrame: 30


In [ ]:
print("Column names:")
print(liver_clean_df.columns)

Column names:
Index(['N_Days', 'Age', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper',
       'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin',
       'Status_C', 'Status_CL', 'Status_D', 'Drug_D-penicillamine',
       'Drug_Placebo', 'Sex_F', 'Sex_M', 'Ascites_N', 'Ascites_Y',
       'Hepatomegaly_N', 'Hepatomegaly_Y', 'Spiders_N', 'Spiders_Y', 'Edema_N',
       'Edema_S', 'Edema_Y', 'Stage_1', 'Stage_2', 'Stage_3'],
      dtype='object')


In [ ]:
#Define X and Y
#No need to include axis=1 when selecting columns for Y because selecting columns inherently operates along columns
#'axis=1' only necessary in drop method to specify dropping columns (as opposed to rows, which would be axis=0)
X=liver_clean_df.drop(["Stage_1", "Stage_2", "Stage_3"],axis=1)
Y=liver_clean_df[["Stage_1", "Stage_2","Stage_3"]]

In [ ]:
# Split preprocessed data into training and testing datasets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.2, random_state=78)

In [ ]:
# Create StandardScaler instances
scaler = StandardScaler()

# Fit StandardScaler
X_scaler = scaler.fit(X_train)

# Scale data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
print(X_train_scaled)

[[ 1.47478759  1.91977093 -0.58030606 ...  0.44623851 -0.39140986
  -0.18534615]
 [-0.54840842  1.50485971  0.74594393 ...  0.44623851 -0.39140986
  -0.18534615]
 [-0.85620447 -0.37584224  1.71852725 ...  0.44623851 -0.39140986
  -0.18534615]
 ...
 [ 0.46882    -0.01137535  0.61331893 ...  0.44623851 -0.39140986
  -0.18534615]
 [ 0.78318487 -0.89745246  0.54700643 ...  0.44623851 -0.39140986
  -0.18534615]
 [ 0.9014236   0.12939338 -0.44768106 ...  0.44623851 -0.39140986
  -0.18534615]]


In [ ]:
print(X_test_scaled)

[[ 2.46104872 -0.93178629 -0.5582019  ...  0.44623851 -0.39140986
  -0.18534615]
 [-1.19684462  0.25537215 -0.27084773 ... -2.24095404  2.55486668
  -0.18534615]
 [ 1.27396943 -0.16350066  0.01650643 ...  0.44623851 -0.39140986
  -0.18534615]
 ...
 [ 2.01343068  1.80567694 -0.6245144  ...  0.44623851 -0.39140986
  -0.18534615]
 [ 1.03561517 -1.13224308 -0.46978523 ...  0.44623851 -0.39140986
  -0.18534615]
 [ 0.12724145 -1.51176403 -0.5582019  ...  0.44623851 -0.39140986
  -0.18534615]]


In [ ]:
#Verify type of X_train_scaled, X_test_scaled to ensure scaling turned them into arrays
print(type(X_train_scaled))
print(type(X_test_scaled))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
#Verify type of Y_test and Y_train
print(type(Y_train))
print(type(Y_test))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [ ]:
# Convert Y_train and Y_test to arrays
Y_train = Y_train.to_numpy()
Y_test = Y_test.to_numpy()

#Verify type of Y_test and Y_train
print(type(Y_train))
print(type(Y_test))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
# Print array shapes
print("Shape of X_train_scaled:", X_train_scaled.shape)
print("Shape of X_test_scaled:", X_test_scaled.shape)
print("Shape of Y_train:", Y_train.shape)
print("Shape of Y_test:", Y_test.shape)

Shape of X_train_scaled: (1927, 27)
Shape of X_test_scaled: (7712, 27)
Shape of Y_train: (1927, 3)
Shape of Y_test: (7712, 3)


In [ ]:
#Verify Y_train and Y_test are one-hot-encoded

def is_one_hot_encoded(array):
    # Check if all elements are 0 or 1
    if np.all(np.logical_or(array == 0, array == 1)):
        # Check if each row contains only one 1 and the rest are 0s
        return np.all(np.sum(array, axis=1) == 1) and np.all(np.sum(array, axis=1) == 1)
    return False

# Test both arrays
print("Y_train is one-hot encoded:", is_one_hot_encoded(Y_train))
print("Y_test is one-hot encoded:", is_one_hot_encoded(Y_test))

Y_train is one-hot encoded: True
Y_test is one-hot encoded: True


Neural Network Model Without Optimization

In [ ]:
#First iterations
# Define model to be deep neural net

#calculate number of input features in dataset based on first row of input features (X_train_scaled)
number_input_features = X_train_scaled.shape[1]
hidden_nodes_layer1 =  X_train_scaled.shape[1]
hidden_nodes_layer2 = X_train_scaled.shape[1]

#Create Keras Sequantial model
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=3, activation="softmax"))

# Check structure of model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 27)                756       
                                                                 
 dense_1 (Dense)             (None, 27)                756       
                                                                 
 dense_2 (Dense)             (None, 3)                 84        
                                                                 
Total params: 1596 (6.23 KB)
Trainable params: 1596 (6.23 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train model
fit_model = nn.fit(X_train_scaled,Y_train,epochs=100)

Epoch 1/100
61/61 [==============================] - 2s 4ms/step - loss: 0.6371 - accuracy: 0.4245
Epoch 2/100
61/61 [==============================] - 0s 3ms/step - loss: 0.5853 - accuracy: 0.4899
Epoch 3/100
61/61 [==============================] - 0s 4ms/step - loss: 0.5644 - accuracy: 0.5278
Epoch 4/100
61/61 [==============================] - 0s 3ms/step - loss: 0.5511 - accuracy: 0.5413
Epoch 5/100
61/61 [==============================] - 0s 3ms/step - loss: 0.5412 - accuracy: 0.5599
Epoch 6/100
61/61 [==============================] - 0s 3ms/step - loss: 0.5328 - accuracy: 0.5610
Epoch 7/100
61/61 [==============================] - 0s 3ms/step - loss: 0.5261 - accuracy: 0.5739
Epoch 8/100
61/61 [==============================] - 0s 3ms/step - loss: 0.5201 - accuracy: 0.5802
Epoch 9/100
61/61 [==============================] - 0s 3ms/step - loss: 0.5144 - accuracy: 0.5817
Epoch 10/100
61/61 [==============================] - 0s 3ms/step - loss: 0.5087 - accuracy: 0.5906
Epoch 11/

In [ ]:
# Evaluate model using test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,Y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

241/241 - 0s - loss: 0.5699 - accuracy: 0.6289 - 385ms/epoch - 2ms/step
Loss: 0.5699119567871094, Accuracy: 0.6288900375366211


In [ ]:
#Second iteration: add more neurons to hidden layers
# Define model to be deep neural net

#calculate number of input features in dataset based on first row of input features (X_train_scaled)
number_input_features = (X_train_scaled.shape[1])
hidden_nodes_layer1 =  (X_train_scaled.shape[1])*2
hidden_nodes_layer2 = (X_train_scaled.shape[1])*2

# Create Keras Sequential model
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=3, activation="softmax"))

# Check structure of model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 54)                1512      
                                                                 
 dense_7 (Dense)             (None, 54)                2970      
                                                                 
 dense_8 (Dense)             (None, 3)                 165       
                                                                 
Total params: 4647 (18.15 KB)
Trainable params: 4647 (18.15 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile model with accuracy metric
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train model
fit_model = nn.fit(X_train_scaled,Y_train,epochs=100)

Epoch 1/100
61/61 [==============================] - 1s 2ms/step - loss: 0.6147 - accuracy: 0.4499
Epoch 2/100
61/61 [==============================] - 0s 2ms/step - loss: 0.5545 - accuracy: 0.5439
Epoch 3/100
61/61 [==============================] - 0s 2ms/step - loss: 0.5346 - accuracy: 0.5739
Epoch 4/100
61/61 [==============================] - 0s 2ms/step - loss: 0.5213 - accuracy: 0.5885
Epoch 5/100
61/61 [==============================] - 0s 2ms/step - loss: 0.5116 - accuracy: 0.5994
Epoch 6/100
61/61 [==============================] - 0s 2ms/step - loss: 0.5009 - accuracy: 0.6108
Epoch 7/100
61/61 [==============================] - 0s 2ms/step - loss: 0.4919 - accuracy: 0.6258
Epoch 8/100
61/61 [==============================] - 0s 2ms/step - loss: 0.4833 - accuracy: 0.6326
Epoch 9/100
61/61 [==============================] - 0s 2ms/step - loss: 0.4740 - accuracy: 0.6373
Epoch 10/100
61/61 [==============================] - 0s 2ms/step - loss: 0.4662 - accuracy: 0.6565
Epoch 11/

In [ ]:
# Evaluate model using test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,Y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

241/241 - 1s - loss: 0.7411 - accuracy: 0.6505 - 778ms/epoch - 3ms/step
Loss: 0.7411323189735413, Accuracy: 0.6505445837974548


In [ ]:
#Third iteration: Add more hidden layers
# Define model to be deep neural net

# Create Keras Sequential model
nn = tf.keras.models.Sequential()

#calculate number of input features in dataset based on first row of input features (X_train_array)
number_input_features = (X_train_scaled.shape[1])
hidden_nodes_layer1 =  (X_train_scaled.shape[1])*2
hidden_nodes_layer2 = (X_train_scaled.shape[1])*2
hidden_nodes_layer3 = (X_train_scaled.shape[1])*2
hidden_nodes_layer4 = (X_train_scaled.shape[1])*2

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=3, activation="softmax"))

# Check structure of model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 54)                1512      
                                                                 
 dense_10 (Dense)            (None, 54)                2970      
                                                                 
 dense_11 (Dense)            (None, 54)                2970      
                                                                 
 dense_12 (Dense)            (None, 54)                2970      
                                                                 
 dense_13 (Dense)            (None, 3)                 165       
                                                                 
Total params: 10587 (41.36 KB)
Trainable params: 10587 (41.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile model with accuracy metric
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train model
fit_model = nn.fit(X_train_scaled,Y_train,epochs=100)

Epoch 1/100
61/61 [==============================] - 1s 2ms/step - loss: 0.6123 - accuracy: 0.4530
Epoch 2/100
61/61 [==============================] - 0s 2ms/step - loss: 0.5480 - accuracy: 0.5490
Epoch 3/100
61/61 [==============================] - 0s 2ms/step - loss: 0.5189 - accuracy: 0.5843
Epoch 4/100
61/61 [==============================] - 0s 3ms/step - loss: 0.5014 - accuracy: 0.5957
Epoch 5/100
61/61 [==============================] - 0s 3ms/step - loss: 0.4839 - accuracy: 0.6160
Epoch 6/100
61/61 [==============================] - 0s 3ms/step - loss: 0.4659 - accuracy: 0.6424
Epoch 7/100
61/61 [==============================] - 0s 3ms/step - loss: 0.4486 - accuracy: 0.6544
Epoch 8/100
61/61 [==============================] - 0s 3ms/step - loss: 0.4296 - accuracy: 0.6777
Epoch 9/100
61/61 [==============================] - 0s 3ms/step - loss: 0.4273 - accuracy: 0.6762
Epoch 10/100
61/61 [==============================] - 0s 3ms/step - loss: 0.4068 - accuracy: 0.7063
Epoch 11/

In [ ]:
# Evaluate model using test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,Y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

241/241 - 0s - loss: 1.6261 - accuracy: 0.6342 - 401ms/epoch - 2ms/step
Loss: 1.6261131763458252, Accuracy: 0.6342064142227173
